In [4]:
from pathlib import Path
import pandas as pd

In [12]:
# Set path to CSV and read in CSV
df1=pd.read_csv('/work/Fintech-Project2/TRADE_STRATS/XRP_MACD_TRADE_STRATEGY', index_col = 'Unnamed: 0', infer_datetime_format= True)
df1.set_index(pd.to_datetime(df1['Timestamp'], infer_datetime_format=True), inplace=True)
df1.drop(columns=['Timestamp'], inplace=True)
df2=pd.read_csv('/work/Fintech-Project2/TRADE_STRATS/XRP AROON 25 TRADE STRATEGY.CSV')
df2.set_index(pd.to_datetime(df2['Timestamp'], infer_datetime_format=True), inplace=True)
df2.drop(columns=['Timestamp'], inplace=True)
df3=pd.read_csv('/work/Fintech-Project2/TRADE_STRATS/XRP_RSI_TRADE_STRATEGY')
df3.set_index(pd.to_datetime(df3['Timestamp'], infer_datetime_format=True), inplace=True)
df3.drop(columns=['Timestamp'], inplace=True)
df = pd.concat([df1],[df2],[df3], join = 'inner')


KeyError: 'Timestamp'

In [18]:
df1=pd.read_csv('/work/Fintech-Project2/TRADE_STRATS/XRP_MACD_TRADE_STRATEGY')
df1.set_index(pd.to_datetime(df1['Timestamp'], infer_datetime_format=True), inplace=True)
df1.drop(columns=['Timestamp'], inplace=True)

KeyError: 'Timestamp'

In [16]:
df2=pd.read_csv('/work/Fintech-Project2/TRADE_STRATS/XRP AROON 25 TRADE STRATEGY.CSV')
df2

,Unnamed: 0,Close,trend_aroon_up,trend_aroon_down,aroon_signal,aroon_position
0,2019-09-01 01:00:00+01:00,0.259100,NaN,NaN,0,1
1,2019-09-01 02:00:00+01:00,0.258400,NaN,NaN,0,1
2,2019-09-01 03:00:00+01:00,0.257800,NaN,NaN,0,1
3,2019-09-01 04:00:00+01:00,0.258500,NaN,NaN,0,1
4,2019-09-01 05:00:00+01:00,0.258100,NaN,NaN,0,1
...,...,...,...,...,...,...
16999,2021-08-11 00:00:00+01:00,0.851247,92.0,20.0,0,1
17000,2021-08-11 01:00:00+01:00,0.869300,100.0,16.0,0,1
17001,2021-08-11 02:00:00+01:00,0.873887,100.0,12.0,0,1
17002,2021-08-11 03:00:00+01:00,0.875611,100.0,8.0,0,1


In [17]:
df3=pd.read_csv('/work/Fintech-Project2/TRADE_STRATS/XRP_RSI_TRADE_STRATEGY')
df3

,Datetime,Close,RSI,rsi_signal,rsi_position
0,2019-09-01 01:00:00+01:00,0.259100,NaN,0,1
1,2019-09-01 02:00:00+01:00,0.258400,0.000000,-1,0
2,2019-09-01 03:00:00+01:00,0.257800,0.000000,0,0
3,2019-09-01 04:00:00+01:00,0.258500,7.211196,0,0
4,2019-09-01 05:00:00+01:00,0.258100,6.904779,0,0
...,...,...,...,...,...
16997,2021-08-10 22:00:00+01:00,0.858775,66.758844,0,1
16998,2021-08-10 23:00:00+01:00,0.855940,65.019373,0,1
16999,2021-08-11 00:00:00+01:00,0.851247,62.132825,0,1
17000,2021-08-11 01:00:00+01:00,0.869159,67.975899,0,1


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d12c285c-6789-4c45-9ad7-1be9b4d60622' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>